In [1]:
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

In [2]:
# Чтение файла
df = pd.read_csv('./data/train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Исключение признаков, которые на ваш взгляд, могут привести к переобучению

In [3]:
# Исключение признаков
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

## Разделение выборки на train и test

In [4]:
# Разделение выборки на train и test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [5]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,0,1,male,45.5,0,0,28.5000,S
733,0,2,male,23.0,0,0,13.0000,S
382,0,3,male,32.0,0,0,7.9250,S
704,0,3,male,26.0,1,0,7.8542,S
813,0,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...,...
106,1,3,female,21.0,0,0,7.6500,S
270,0,1,male,NaN,0,0,31.0000,S
860,0,3,male,41.0,2,0,14.1083,S
435,1,1,female,14.0,1,2,120.0000,S


In [6]:
test_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,1,3,male,NaN,1,1,15.2458,C
439,0,2,male,31.0,0,0,10.5000,S
840,0,3,male,20.0,0,0,7.9250,S
720,1,2,female,6.0,0,1,33.0000,S
39,1,3,female,14.0,1,0,11.2417,C
...,...,...,...,...,...,...,...,...
433,0,3,male,17.0,0,0,7.1250,S
773,0,3,male,NaN,0,0,7.2250,C
25,1,3,female,38.0,1,5,31.3875,S
84,1,2,female,17.0,0,0,10.5000,S


## Преобразование категориальных признаков с помощью sklearn.preprocessing.OneHotEncoder, а некатегориальных с помощью sklearn.preprocessing.StandardScaler

In [7]:
# Преобразование категориальных признаков
cat_features = ['Sex', 'Embarked']
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_df[cat_features])
train_cat = enc.transform(train_df[cat_features]).toarray()
test_cat = enc.transform(test_df[cat_features]).toarray()

In [8]:
# Преобразование некатегориальных признаков
num_features = ['Age', 'SibSp', 'Parch', 'Fare']
scaler = StandardScaler()
scaler.fit(train_df[num_features])
train_num = scaler.transform(train_df[num_features])
test_num = scaler.transform(test_df[num_features])

In [9]:
# Объединение преобразованных признаков
train_X = np.concatenate((train_num, train_cat), axis=1)
test_X = np.concatenate((test_num, test_cat), axis=1)
train_y = train_df['Survived']
test_y = test_df['Survived']

## Обучение логистической регрессии и KNN моделей и расчет всех метрик классификации

In [10]:
# Обучение модели логистической регрессии
lr = LogisticRegression(random_state=42)
train_X = pd.DataFrame(train_X) # преобразуем train_X в объект pandas DataFrame
train_X.fillna(train_X.mean(), inplace=True) # используем метод fillna() из pandas

lr.fit(train_X, train_y)
test_X = pd.DataFrame(test_X) # преобразуем test_X в объект pandas DataFrame
test_X.fillna(test_X.mean(), inplace=True) # используем метод fillna() из pandas
lr_preds = lr.predict(test_X)

In [11]:
# Расчет метрик для логистической регрессии
print('Logistic Regression')
print(f'Accuracy: {accuracy_score(test_y, lr_preds)}')
print(f'Precision: {precision_score(test_y, lr_preds)}')
print(f'Recall: {recall_score(test_y, lr_preds)}')
print(f'F1 score: {f1_score(test_y, lr_preds)}')
print(f'ROC AUC score: {roc_auc_score(test_y, lr_preds)}\n')

Logistic Regression
Accuracy: 0.7932960893854749
Precision: 0.7846153846153846
Recall: 0.6891891891891891
F1 score: 0.7338129496402878
ROC AUC score: 0.7779279279279279



In [12]:
# Обучение модели KNN
knn = KNeighborsClassifier()
knn.fit(train_X, train_y)
knn_preds = knn.predict(test_X)

In [13]:
# Расчет метрик для KNN
print('KNN')
print(f'Accuracy: {accuracy_score(test_y, knn_preds)}')
print(f'Precision: {precision_score(test_y, knn_preds)}')
print(f'Recall: {recall_score(test_y, knn_preds)}')
print(f'F1 score: {f1_score(test_y, knn_preds)}')
print(f'ROC AUC_score: {roc_auc_score(test_y, knn_preds)}')

KNN
Accuracy: 0.7877094972067039
Precision: 0.7647058823529411
Recall: 0.7027027027027027
F1 score: 0.7323943661971832
ROC AUC_score: 0.7751608751608752


## Подбор гиперпараметров для обоих моделей с помощью RandomizedSearchCV

In [14]:
# Подбор гиперпараметров для модели логистической регрессии
logreg_params = {
'C': uniform(0.01, 100)
}

logreg = LogisticRegression()
logreg_search = RandomizedSearchCV(logreg, logreg_params, n_iter=100, cv=5, random_state=42)
logreg_search.fit(train_X, train_y)

print('Best params for Logistic Regression:', logreg_search.best_params_)
print('Best score for Logistic Regression:', logreg_search.best_score_)

Best params for Logistic Regression: {'C': 37.464011884736244}
Best score for Logistic Regression: 0.7920811582783414


In [15]:
# Подбор гиперпараметров для модели KNN
knn_params = {
'n_neighbors': randint(1, 30),
'p': randint(1, 3)
}

knn_search = RandomizedSearchCV(knn, knn_params, n_iter=100, cv=5, random_state=42)
knn_search.fit(train_X, train_y)

print('Best params for KNN:', knn_search.best_params_)
print('Best score for KNN:', knn_search.best_score_)

Best params for KNN: {'n_neighbors': 16, 'p': 1}
Best score for KNN: 0.817393873731902


## Почему, как вы думаете, эти параметры подошли?

Параметры считаются подходящими, потому что они были выбраны на основе подбора гиперпараметров с использованием RandomizedSearchCV, который позволяет выбирать параметры случайным образом из заданных диапазонов и находить наилучшие значения для модели.

Для модели логистической регрессии, лучшее значение параметра C было выбрано как 37.46, а для модели KNN лучшее значение параметра n_neighbors было выбрано как 16 и параметр p был выбран как 1.

Для обеих моделей, значение метрики best_score (лучшего показателя) достаточно высоко, чтобы считать параметры подходящими для использования в модели. Это означает, что модель достаточно хорошо работает на тестовых данных и может предсказывать классы с высокой точностью.

## Расчет метрик для новых моделей и вывод

In [16]:
# Оценка лучшей модели логистической регрессии
lr_best_model = logreg_search.best_estimator_
lr_best_model.fit(train_X, train_y)
lr_y_pred = lr_best_model.predict(test_X)

print("Метрики для лучшей модели логистической регрессии:")
print(classification_report(test_y, lr_y_pred))

# Оценка лучшей модели KNN
knn_best_model = knn_search.best_estimator_
knn_best_model.fit(train_X, train_y)
knn_y_pred = knn_best_model.predict(test_X)

print("Метрики для лучшей модели KNN:")
print(classification_report(test_y, knn_y_pred))

Метрики для лучшей модели логистической регрессии:
              precision    recall  f1-score   support

           0       0.80      0.87      0.83       105
           1       0.78      0.69      0.73        74

    accuracy                           0.79       179
   macro avg       0.79      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179

Метрики для лучшей модели KNN:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       105
           1       0.79      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179



### Выводы:

Исходя из результатов, можно сделать вывод, что обе модели достаточно хорошо справляются с классификацией пассажиров на выживших и не выживших на основе предоставленных данных. Модель KNN показала немного лучший результат по метрике f1-score (0.75 против 0.73 у модели логистической регрессии), однако точность и полнота (precision и recall) у обеих моделей достаточно близки. Следовательно, обе модели можно считать подходящими для решения поставленной задачи.